<a href="https://colab.research.google.com/github/Kaumini-Perera/my-project/blob/main/Mapping_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# First we will import the earth engine package
import ee

# Trigger the authentication
ee.Authenticate()

# Initialize the library
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=5xei-tT9Fp_P5LcBrbdVJSoDmZLVQ9dBkHmTy26-k68&tc=MG8OMPQt9grzzcl3egqz7_9SRix6TKuCBAiG1Ru0VVM&cc=JfQVK454lkYS-uScZ2r1hnaCPMra-Af_JXGtbqUHDD8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXk34V2djMOUkgoSGtgfdQVfaiMNXScERbTj9V5rVk4wZwKWz0pWpOA

Successfully saved authorization token.


In [4]:
# Import the feature collection as the world dataset
world = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
# Map over the feature collection to extract the 'country_na' attribute and get unique values
country_names = world.aggregate_array('country_na').distinct().getInfo()
# Print the list of country names
for country in country_names:
  print (country)
  #the country that i chosen is "Italy"

Chad
Malawi
Zambia
Zimbabwe
Botswana
Namibia
Angola
Burundi
Rwanda
South Africa
Lesotho
Swaziland
Mayotte
Niger
Sudan
Libya
Koualou Area
Tunisia
Egypt
Bir Tawil
Halaib Triangle
Eritrea
Ethiopia
Djibouti
Somalia
South Sudan
Abyei Area
Kenya
Uganda
Tanzania
Mozambique
Comoros
Madagascar
Cabo Verde
Western Sahara
Mauritania
Morocco
Spain (Canary Is)
Portugal (Madeira Is)
Spain (Africa)
Algeria
Mali
Burkina Faso
Togo
Ghana
Cote d'Ivoire
Guinea
Senegal
Gambia, The
Guinea-Bissau
Sierra Leone
Liberia
Benin
Nigeria
Cameroon
Equatorial Guinea
Sao Tome & Principe
Gabon
Rep of the Congo
Central African Rep
Dem Rep of the Congo
Hong Kong
Macau
Taiwan
Senkakus
Japan
Korea, South
Korea, North
Korean Is. (UN Jurisdiction)
China
Liancourt Rocks
Russia
Ukraine
Romania
Moldova
Finland
Norway
Andorra
Italy
Monaco
Malta
San Marino
Vatican City
Croatia
Albania
Bosnia & Herzegovina
Montenegro
Serbia
Greece
Macedonia
Kosovo
Bulgaria
Portugal (Azores)
Portugal
Gibraltar
Spain
France
Svalbard
Sweden
Denmark
Es

In [12]:
#import the folim library
import folium

#Reducing the feature collection
#area of interest -> aoi
country = 'Italy'
aoi = world.filter(ee.Filter.eq('country_na', country))

#load an image of dailt precipitation in mm/day
#sums up the precipitation  values from all the images in the filtered image collection.
#the result is a single image file.
#the total accumulated precipitation for the entire year in 2017 within the AOI.
precip = (ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').select('precipitation').filterDate('2017-01-01', '2017-12-31').filterBounds(aoi).sum())

# Define a colour palette to visualize the images
RF_PALETTE = "ff0000, ffff00, 00ffff, 0000ff"
vis_params = {'min': 0,'max': 2000,'palette': RF_PALETTE}

In [13]:
# Function to visualize Earth Engine image layers using folium.
def add_ee_layer(self, ee_object, vis_params, name):
  try:
    # display ee.Image()
    if isinstance(ee_object, ee.image.Image):
      map_id_dict = ee.Image(ee_object).getMapId(vis_params)
      folium.raster_layers.TileLayer(tiles = map_id_dict['tile_fetcher'].url_format,attr =
                                     'Google Earth Engine',name = name,overlay = True,
                                     control = True).add_to(self)

    # display ee.ImageCollection()
    elif isinstance(ee_object, ee.imagecollection.ImageCollection):
      ee_object_new = ee_object.mosaic()
      map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
      folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Google Earth Engine', name = name, overlay = True, control = True).add_to(self)

    # display ee.Geometry()
    elif isinstance(ee_object, ee.geometry.Geometry):
      folium.GeoJson(data = ee_object.getInfo(), name = name, overlay = True,
                     control = True).add_to(self)

    # display ee.FeatureCollection()
    elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
      ee_object_new = ee.Image().paint(ee_object, 0, 2)
      map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
      folium.raster_layers.TileLayer(tiles = map_id_dict['tile_fetcher'].url_format,
                                     attr = 'Google Earth Engine', name = name,
                                     overlay = True, control = True).add_to(self)

  except:
    print("Could not display {}".format(name))

In [15]:
# Add custom method to folium (now both folium and earth engine)
folium.Map.add_ee_layer = add_ee_layer

# Create a map centered around the country
location = [41.87194, 12.56738]
map_country = folium.Map(location=location, zoom_start=7)
map_country.add_ee_layer(precip.clip(aoi), vis_params, 'Rainfall')
map_country.add_ee_layer(aoi, {}, country)

# Display the map
display(map_country)

In [16]:
# Define the years for which you want to get the precip images
years = [2010, 2011, 2012, 2013, 2014, 2015]
# Initialize an empty dictionary to store the precip images
precip_images = {}
# Loop through each year
for year in years:
  # Convert the year to string for date formatting
  str_year = str(year)
  # Load an image of daily precipitation in mm/day.
  # Sums up the precipitation values from all the images in the filtered ImageCollection.
  # The result is a single ee.Image where each pixel value represents
  # the total accumulated precipitation for the entire year within the AOI.
  precip = (ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD').select('precipitation').filterDate(f'{str_year}-01-01', f'{str_year}-12-31').filterBounds(aoi).sum())
  # Store the precip image in the dictionary using the year as the key
  precip_images[year] = precip

In [17]:
# Add the precipitation layers for each year
for year in [2010, 2011, 2012, 2013, 2014, 2015]:
  precip = precip_images[year]
  map_country.add_ee_layer(precip.clip(aoi), vis_params, f'Rainfall {year}')

# Add the AOI layer
map_country.add_ee_layer(aoi, {}, country)

# Add Layer Control
folium.LayerControl().add_to(map_country)

# Display the map
display(map_country)

In [18]:
# Define the years for which you want to get the NDVI images
years = [2010, 2011, 2012, 2013, 2014, 2015]

# Initialize an empty dictionary to store the NDVI images
ndvi_images = {}

# Loop through each year
for year in years:
  # Convert the year to string for date formatting
  str_year = str(year)
  # Load the image collection for the given year within the AOI.
  # The MODIS MOD09Q1 product has a band named 'sur_refl_b02'
  # for red and 'sur_refl_b01' for NIR.
  images = (ee.ImageCollection('MODIS/006/MOD09Q1').select(['sur_refl_b01', 'sur_refl_b02']).filterDate(f'{str_year}-01-01', f'{str_year}-12-31').filterBounds(aoi))
  # Compute the mean NDVI for the entire year.
  # NDVI formula: (NIR - RED) / (NIR + RED)
  mean_ndvi = images.map(lambda img:
    img.normalizedDifference(['sur_refl_b01','sur_refl_b02'])).mean()
  # Store the mean NDVI image in the dictionary using the year as the key
  ndvi_images[year] = mean_ndvi

# Now ndvi_images contains the mean NDVI images for 2017, 2018, and 2019
# Add custom method to folium
folium.Map.add_ee_layer = add_ee_layer

# Define a colour palette to visualize the images
RF_PALETTE = "ff0000, ffff00, 00ffff, 0000ff"
vis_params = {'min': -1,'max': 1,'palette': RF_PALETTE}

In [20]:
# Create a map centered around the country
location = [41.87194, 12.56738]
map_country = folium.Map(location=location, zoom_start=7)

# Add the ndvi layers for each year
for year in [2010, 2011, 2012, 2013, 2014, 2015]:
  ndvi = ndvi_images[year]
  map_country.add_ee_layer(ndvi.clip(aoi), vis_params, f'NDVI {year}')

# Add the AOI layer
map_country.add_ee_layer(aoi, {}, country)

# Add Layer Control
folium.LayerControl().add_to(map_country)

# Display the map
display(map_country)

In [21]:
# Assuming the ee library has been imported and initialized
# Assume 'ndvi_2015' is the NDVI image for the year
ndvi_2015 = ndvi_images[2015] # Replace this with the NDVI image for your selected country in 2015

# Set visualization parameters
vis_params = {'min': -1,'max': 1,'palette': RF_PALETTE}

# Get the thumbnail URL
thumbnail_url = ndvi_2015.getThumbUrl({
'bands': 'nd', # This assumes 'nd' is the band name for NDVI; adjust as needed
'min': vis_params['min'],'max': vis_params['max'],'palette': vis_params['palette'],
                                        'region': aoi.geometry().bounds().getInfo()
                                        ['coordinates'],'dimensions': 512,'format': 'png'})

from IPython.display import display, Image
display(Image(url=thumbnail_url))